## 영어 토큰화

In [2]:
en_text = "A Dog Run back conrner near spare bedrooms"

#### 1. spaCy

In [4]:
import spacy
spacy_en = spacy.load('en_core_web_sm')

In [5]:
def tokenize(en_text):
    return [tok.text for tok in spacy_en.tokenizer(en_text)]

In [6]:
print(tokenize(en_text))

['A', 'Dog', 'Run', 'back', 'conrner', 'near', 'spare', 'bedrooms']


#### 2. NLTK

In [3]:
from nltk.tokenize import word_tokenize
print(word_tokenize(en_text))

['A', 'Dog', 'Run', 'back', 'conrner', 'near', 'spare', 'bedrooms']


#### 3. 띄어쓰기 토큰화

In [4]:
print(en_text.split(' '))

['A', 'Dog', 'Run', 'back', 'conrner', 'near', 'spare', 'bedrooms']


## 한국어 토큰화

#### 1. 띄어쓰기 토큰화

In [5]:
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지랑 사왔어."

In [9]:
# '사과' 단어가 4번 등장했는데 조사로 인해 다 다른 단어로 구분

print(kor_text.split())

['사과의', '놀라운', '효능이라는', '글을', '봤어.', '그래서', '오늘', '사과를', '먹으려고', '했는데', '사과가', '썩어서', '슈퍼에', '가서', '사과랑', '오렌지랑', '사왔어.']


In [10]:
len(kor_text.split())

17

#### 형태소 토큰화

In [14]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: C:\Users\dltkd\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: C:\Users\dltkd\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
������ ������ ã�� �� �����ϴ�.


In [15]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(kor_text))

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/